In [ ]:
!gdown --id 1zu97TwzyU2T8OVvQ2VMof-XqHDX_fxPf

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1zu97TwzyU2T8OVvQ2VMof-XqHDX_fxPf
To: /content/data_merged.csv
100% 227M/227M [00:01<00:00, 210MB/s]


In [ ]:
import pandas as pd
import numpy as np

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from sklearn.linear_model import (
    LinearRegression, 
    SGDRegressor, 
    LogisticRegression
)
from sklearn.svm import LinearSVC

# from sklearn.preprocessing import normalize
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

SEED = 42
TEST_SIZE = 0.2

In [ ]:
data = pd.read_csv('./data_merged.csv')

In [ ]:
def get_filtered_X(X, relevant_columns):
    non_relevant_cols = [col for col in data.columns if col not in relevant_columns]
    
    for col in non_relevant_cols:
        X = X.drop(col, axis=1)
    
    return X.to_numpy()

In [ ]:
import re
import ast

for i in range(len(data['crew'])): 
  names = ""
  try:
    data['crew'][i] = data['crew'][i].replace("[","").replace("]","").replace("},","} -")
    x = data['crew'][i].split(" - ")
    for j in x:
      try:
        j = ast.literal_eval(j)
        names = names + j['name'].lower() + ":"
      except:
        pass
    data['crew'][i] = names
  except:
    data['crew'][i] = ""

for i in range(len(data['cast'])): 
  names = ""
  try:
    data['cast'][i] = data['cast'][i].replace("[","").replace("]","").replace("},","} -")
    x = data['cast'][i].split(" - ")
    for j in x:
      try:
        j = ast.literal_eval(j)
        names = names + j['name'].lower() + ":"
      except:
        pass
    data['cast'][i] = names
  except:
    data['cast'][i] = ""

for i in range(len(data['genres'])): 
  names = ""
  try:
    data['genres'][i] = data['genres'][i].replace("[","").replace("]","").replace("},","} -")
    x = data['genres'][i].split(" - ")
    for j in x:
      try:
        j = ast.literal_eval(j)
        names = names + j['name'].lower() + ":"
      except:
        pass
    data['genres'][i] = names
  except:
    data['genres'][i] = ""
    


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.p

In [ ]:
Cast = []
Crew = []
Genre = []
def extract_feats():
  top_directors = ['quentin tarantino', 'christopher nolan', 'martin scorsese', 'steven spielberg']
  top_genres = ['adventure', 'science fiction', 'thriller', 'drama', 'action', 'fantasy']
  # genre_matrix = {'adventure' : 0.8, 'science fiction' : 0.55, 'thriller' : 0.6, 'drama': 0.7, 'action' : 0.5 , 'fantasy': 0.65}
  top_actors = ['leonardo dicaprio', 'tom hanks', 'tom cruise', 'jack nicholson', 'christian bale', 'brad pitt', 'al pacino']
  

  for i in data['cast']:
    i = i.split(':')
    cast = 0
    for j in i:
      if j in top_actors:
        cast = 1
        break
    Cast.append(cast)
  for i in data['crew']:
    i = i.split(':')
    crew = 0
    for j in i:
      if j in top_directors:
        crew = 1
        break
    Crew.append(crew)
  for i in data['genres']:
    i = i.split(':')
    genre = 0
    for j in i:
      if j in top_genres:
        genre = 1
        break
    Genre.append(genre)

extract_feats()
data = data.fillna(0)
data['release_date'] = data['release_date'].apply(
    lambda x: 2022 - int(str(x)[:4])
)

Revenue = list(data['revenue'])  
Budget = list(data['budget'])
Runtime = list(data['runtime'])
Popularity = list(data['popularity'])
Release_date = list(data['release_date'])



In [ ]:
#normalize

r_mean = np.mean(np.array(Revenue))
r_max = np.max(np.array(Revenue))

for i in range(len(Revenue)):
  Revenue[i] = (Revenue[i] - r_mean)/r_max


r_mean = np.mean(np.array(Budget))
r_max = np.max(np.array(Budget))

for i in range(len(Budget)):
  Budget[i] = (Budget[i] - r_mean)/r_max


In [ ]:
X = np.array(list(zip(Revenue,Budget,Genre,Cast,Crew, Runtime,Popularity)))
y = np.array(list(data['vote_average']))
print(y.shape)

(46628,)


In [ ]:
def process(X, y):
    new_X, new_y = [], []
    
    for Xi, yi in zip(X, y):
        if Xi[0] <= 0 or Xi[1] <= 0: continue

        new_X.append(Xi)
        new_y.append(yi)

    return new_X, new_y

X, y = process(X, y)
y = np.array(y)
print(len(X), len(y))

3399 3399


In [ ]:

# rb = [float(i/j) for i,j,k,l,m,n,o,p in X]
# Genre = [k for i,j,k,l,m,n,o,p in X]
# Cast = [l for i,j,k,l,m,n,o,p in X]
# Crew = [m for i,j,k,l,m,n,o,p in X]
# Runtime = [n for i,j,k,l,m,n,o,p in X]
# Popularity = [o for i,j,k,l,m,n,o,p in X]
# Release_date = [p for i,j,k,l,m,n,o,p in X]

# X = np.array(list(zip(rb,Genre,Cast,Crew, Runtime,Popularity,Release_date,)))

# print(X.shape)


(5458, 7)


In [ ]:
def regression(X, y, choice='lr'):
    X_train, X_test, y_train, y_test = train_test_split(
       X, y, test_size=TEST_SIZE, random_state=SEED
    )
    regressor = None
    if choice == 'lr':
        regressor = LinearRegression()
    elif choice == 'sgd':
        regressor = SGDRegressor(max_iter=1000, tol=1e-3)
    else:
        raise NotImplementedError

    model = make_pipeline(StandardScaler(), regressor)
    model.fit(X_train, y_train)
    print(f"{choice} score: {model.score(X_test, y_test)}")
    return

In [ ]:
regression(X, y, 'lr')
regression(X, y, 'sgd')

lr score: 0.2799244941397596
sgd score: 0.27490078190351974


In [ ]:
def convert_to_categorical(y):
    y = list(map(int, y))
    y = to_categorical(y)
    return y

def classification(X, y, choice='lr'):
    y = list(map(int, y))

    X_train, X_test, y_train, y_test = train_test_split(
       X, y, test_size=TEST_SIZE, random_state=SEED
    )

    classifier = None
    if choice == 'lr':
        classifier = LogisticRegression(max_iter=5000)
    elif choice == 'svc':
        classifier = LinearSVC(max_iter=5000)
    else:
        raise NotImplementedError

    model = make_pipeline(StandardScaler(), classifier)
    model.fit(X_train, y_train)
    print(f"{choice} score: {model.score(X_test, y_test)}")
    return model

In [ ]:
lr = classification(X, y, 'lr')
svc = classification(X, y, 'svc')

lr score: 0.5367647058823529
svc score: 0.5352941176470588


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
from joblib import dump

dump(lr, 'logreg.joblib')
dump(svc, 'svc.joblib')

['svc.joblib']